------------------------------------------------------------------------------------------------------------------------------------------------------

## CALIBRATION NOTEBOOK

------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import sys; sys.path.insert(0, '../'); from lib import *;

In [ ]:
# Set options for general visualitation
OPT  = {
    "MICRO_SEC":   True,                # Time in microseconds (True/False)
    "NORM":        False,               # Runs can be displayed normalised (True/False)
    "ALIGN":       True,                # Aligns waveforms in peaktime (True/False)
    "LOGY":        False,               # Runs can be displayed in logy (True/False)
    "SHOW_AVE":    "",                  # If computed, vis will show average (AveWvf,AveWvfSPE,etc.)
    "SHOW_PARAM":  False,               # Print terminal information (True/False)
    "CHARGE_KEY":  "ChargeAveRange",    # Select charge info to be displayed. Default: "ChargeAveRange" (if computed)
    "PEAK_FINDER": False,               # Finds possible peaks in the window (True/False)
    "LEGEND":      True,                # Shows plot legend (True/False)
    "SHOW":        False,
    "THRESHOLD":  100,
    "WIDTH":      5,
    "PROMINENCE": 0.3,
    "ACCURACY":   2000,
    "STYLE":       "CIEMAT_style",      # Plot style. Default: "CIEMAT_style"
    "TERMINAL_MODE": False
    }

style_selector(OPT)

The following cell can be used to load the configuration file.
Check that everything is OK and choose your runs!

In [ ]:
info = read_input_file("TUTORIAL")     # Read input file
for key in info: print(key,info[key])  # Print loaded info
for key in info: print(key,info[key])  # Print loaded info

In [ ]:
chs  = [0,6]
runs = [1] 
my_runs = load_npy(np.asarray(runs).astype(int),np.asarray(chs).astype(int),preset="EVA",info=info,compressed=True) # LOAD YOUR RUNS

In [ ]:
RUN2PLOT = "CALIB_RUNS"; ch = 6; run = 1
xvar = "AnaPeakAmp"
yvar = "AnaChargeAveRangeSIGNAL"
plt.ioff()
for c,ch in enumerate(chs):
    plt.figure()
    plt.hist2d(my_runs[run][ch][xvar],
               my_runs[run][ch][yvar],
               bins=1000,density=True, cmap = viridis, norm=LogNorm())
    plt.xlim([-10,200])
    plt.ylim([-100,10000])
    plt.xlabel(xvar)
    plt.ylabel(yvar)
    plt.colorbar()
    plt.ion()
    plt.show()

In [ ]:
OPT["THRESHOLD"]  = 100
OPT["WIDTH"]      = 8
OPT["PROMINENCE"] = 0.1
OPT["ACCURACY"]   = 1800

In [ ]:
params = {"THRESHOLD": 10, "WIDTH": 15, "PROMINENCE": 0.5, "ACCURACY": 500, "FIT": "gaussian"}
new_params = {}
for i,param in enumerate(params.keys()):
    if check_key(OPT,param) == True: new_params[param] = OPT[param]
    else:                            new_params[param] = params[param]
print(new_params)

In [ ]:
def fit_gaussians(x, y, *p0):
    assert x.shape == y.shape, "Input arrays must have the same shape."
    # try:
    popt, pcov = curve_fit(gaussian_train, x,y, p0=p0[0], maxfev=8000)
    fit_y=gaussian_train(x,*popt)
    chi_squared = np.sum((y[abs(fit_y)>0.1] - fit_y[abs(fit_y)>0.1]) ** 2 / fit_y[abs(fit_y)>0.1]) / (y.size - len(popt))
    return popt,fit_y,chi_squared

In [ ]:
keys = ["AnaChargeAveRangeSIGNAL"]; 
density = OPT["NORM"]
binning = OPT["ACCURACY"]
debug = True
channels = [6]

for r,run in enumerate(runs["CALIB_RUNS"]["NRun"]):
    for c,ch in enumerate(channels):
        fig_cal, ax_cal = plt.subplots(1,1, figsize = (8,6)); add_grid(ax_cal)
        for key in keys:
            # counts, bins, bars = ax_cal.hist( runs["CALIB_RUNS"][run][ch][key],bins = int(binning),histtype="step", label=key, density=density);
            counts, bins, bars = ax_cal.hist( runs["CALIB_RUNS"][run][ch][key],bins = int(binning),histtype="step", label="Data", density=density);
            counts,bins        = np.histogram(runs["CALIB_RUNS"][run][ch][key],int(binning));#need to convert to PE

            peaks = find_peaks(counts,height=new_params["THRESHOLD"],width=new_params["WIDTH"])
            ax_cal.plot(bins[peaks[0]],peaks[1]["peak_heights"],"x") #a Rodrigo le gustan las x para los picos jaja
            ax_cal.set_xlim([-1e3,bins[peaks[0][-1]]*5])
            print("FOUNDED PEAKS:", peaks)

            params=np.zeros(len(peaks[0])*3)
            params[0::3]=peaks[1]["peak_heights"]
            params[1::3]=bins[peaks[0]]
            params[2::3]=1e2
            x=bins[peaks[0][-1]]*1.2
            my_vars,fit_y,qs=fit_gaussians(bins[:-1],counts,params)    
            
            # fit_params, _ = curve_fit(multiple_gaussians, bins[:-1], counts, p0=params, maxfev=8000)
            # plt.plot(bins[:-1], multiple_gaussians(bins[:-1], *fit_params), 'r', label='Fitted Curve')

            my_vars,fit_y,qs=fit_gaussians(bins[:-1],counts,params)     
            plt.plot(bins[:-1], fit_y, 'r-', label='Best Fit',linewidth=1)

        # plt.semilogy()
        fig_cal.suptitle("Run {} Ch {} - Calibration histogram".format(runs["CALIB_RUNS"]["NRun"][0],ch,key))
        # fig_cal.suptitle("Run_{} Ch_{} - {} histogram".format(runs["CALIB_RUNS"]["NRun"][0],ch,key))
        fig_cal.supxlabel("Charge [ADC]"); fig_cal.supylabel("Counts")
        # fig_cal.supxlabel(key + " [ADC]"); fig_cal.supylabel("Counts")
        plt.grid(True, alpha = 0.7)
        plt.legend()
        plt.show()

In [ ]:
popt, pcov, perr = calibrate(runs["CALIB_RUNS"], ["AnaChargeAveRange"], OPT=OPT, debug=False) #TO solve: not repeated plots

## AFTER DECONVOLUTION

In [ ]:
for c, ch in enumerate(channels):
    fast =np.sum(runs["CALIB_RUNS"][runs["CALIB_RUNS"]["NRun"][0]][ch]["DecADC"][:,500:1000],axis=1)
    total=np.sum(runs["CALIB_RUNS"][runs["CALIB_RUNS"]["NRun"][0]][ch]["DecADC"][:,500:3000],axis=1)

    plt.plot(total,fast/total,',')
    plt.ylim([0,1])
    plt.xlim([0,1000])
    CUT=(fast>0.7)*(total>130)
    plt.hist(total[CUT],200,[100,1000]);
    plt.semilogy()
    plt.grid()
    plt.show()

In [ ]:
# With cuts
f90 = fast/total
CUT_total = total>100
CUT_tf90  = f90>0.6
CUT = CUT_tf90*CUT_total

plt.figure(dpi=200)
Amp=np.max(runs["CALIB_RUNS"][runs["CALIB_RUNS"]["NRun"][0]][ch]["DecADC"],axis=1)
plt.plot(total[~CUT], Amp[~CUT], ",");
plt.plot(total[CUT] , Amp[CUT] , ",");
plt.xlim([0,1000])
plt.ylabel("Amp [Arb units]")
plt.xlabel("Charge [Arb units]")
x_ticks=np.arange(0,1000,1)
plt.plot(x_ticks,0.01*x_ticks)
plt.show()

## GAIN FIT

In [ ]:
folder_path = "TUTORIAL"
channels    = [0,6];      run   = 1
my_ov       = [[3,4,5,6],[3]]; debug = False

gains, Dgain = get_gains(run,channels,folder_path=folder_path,debug=debug)
print("STORED GAINS -> ",gains)
print("STORED DGAIN -> ",Dgain)

for c,ch in enumerate(channels):
    filter_gains = np.array(gains[ch])[np.array(gains[ch])!=-99]
    filter_Dgain = np.array(Dgain[ch])[np.array(Dgain[ch])!=-99]
    if filter_gains.size == 0:
        print("No valid gains for channel %i"%ch)
        continue
    a, b = np.polyfit(my_ov[c], filter_gains, 1)
    print("Channel %i: G = %.2f * OV + %.2f"%(ch,a,b))
    plt.errorbar(my_ov[c], filter_gains, yerr=filter_Dgain, label="Channel "+str(ch), marker = "o")

plt.xlabel("OV [V]"); plt.ylabel("Gain [ADCs*ticks]"); plt.title("Gains (run %i)"%run)
plt.grid(True)
plt.legend()
plt.show()
# plt.savefig("gain_fit_SiPMs.png", dpi = 400)